In [594]:
import sys
sys.path.append("..")
import torch
from hbf_layer import HBFLayer
from uci_datasets import Dataset

In [595]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [596]:
data = Dataset("protein")
x_train, y_train, x_test, y_test = data.get_split(split=2)
x_train.shape, x_test.shape

protein dataset, N=45730, d=9


((41157, 9), (4573, 9))

In [597]:
x_train_real = x_train[:36584] #32000 # 2053 13903  36625  42800  36584    36584     39063   13281    2672   # RE-RUN # 13279   # 1279  4699 4701  824   12000
y_train_real = y_train[:36584]
y_train_real = y_train_real.squeeze()
x_val = x_train[36584:]
y_val = y_train[36584:]
y_val = y_val.squeeze()
y_test = y_test.squeeze()

mean = x_train_real.mean(axis=0)
std = x_train_real.std(axis=0)

# x_train_real_normalized = (x_train_real - mean) / std
# x_val_normalized = (x_val - mean) / std
# x_test_normalized = (x_test - mean) / std

x_train_real_normalized = x_train_real
x_val_normalized = x_val
x_test_normalized = x_test



In [598]:
np.random.seed(24)
torch.manual_seed(24) ####################### CHANGE

batch_size = 32 # 64

# X_train, y_train = make_data(n_samples)
# X_test, y_test = X_train, y_train

# x_train, y_train, x_test, y_test

ds_train = torch.utils.data.TensorDataset(torch.from_numpy(x_train_real_normalized).float(), torch.from_numpy(y_train_real).float())
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True) # suffle 

ds_val = torch.utils.data.TensorDataset(torch.from_numpy(x_val_normalized).float(), torch.from_numpy(y_val).float())
dl_val = torch.utils.data.DataLoader(ds_val, batch_size=512, shuffle=False)

ds_test = torch.utils.data.TensorDataset(torch.from_numpy(x_test_normalized).float(), torch.from_numpy(y_test).float())
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=512, shuffle=False)

In [599]:
# Define an RBF layer where the dimensionality of the input feature is 2,
# the number of kernels is 3, and 2 output features

input_dim = 9

# euclidean norm
def euclidean_norm(x):
    return torch.norm(x, p=2, dim=-1)


# Gaussian RBF
def rbf_gaussian(x):
    return (-x.pow(2)).exp()

rbf = HBFLayer(in_features_dim=input_dim,
               num_kernels=1004,
               out_features_dim=1,
               radial_function=rbf_gaussian,
               norm_function=euclidean_norm,
               normalization=False)

In [600]:
for param in rbf.parameters():
    print(param.shape)

torch.Size([1004, 1])
torch.Size([9, 1004])
torch.Size([9, 1004])


In [601]:
total_params = sum(p.numel() for p in rbf.parameters() if p.requires_grad)
total_params

19076

In [602]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(24)
np.random.seed(24)

# Parameters
batch_size = 32
epochs = 100
learning_rate = 1e-3

# Assume pc_rbfn is your model, already defined and initialized elsewhere

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(rbf.parameters(), lr=learning_rate)


nan_counter = 0
exit_loops = False

# Training and Validation Loop
for epoch in range(epochs):
    # Training phase
    rbf.train()
    train_loss = 0.0
    
    for inputs, targets in dl_train:
        optimizer.zero_grad()
        outputs = rbf(inputs)
        outputs = outputs.squeeze(1)  # Ensure outputs match the target's shape
        
        # if(torch.isnan(outputs).any() == False):
        #     print("no NAN")

        if(torch.isnan(outputs).any()):
            print("outputs", outputs)
        
        if(torch.isnan(targets).any()):
            print("targets", targets)
        
        loss = criterion(outputs, targets)
        if(torch.isnan(loss).any()):
            print("loss", loss)
            nan_counter += 1
        if (nan_counter > 3):
            exit_loops = True
            break
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        
    if exit_loops:
        break
    train_loss /= len(dl_train.dataset)

    # Validation phase
    rbf.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in dl_val:
            outputs = rbf(inputs)
            outputs = outputs.squeeze(1)  # Ensure outputs match the target's shape
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
    val_loss /= len(dl_val.dataset)

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")



Epoch 1: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 2: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 3: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 4: Train Loss: 0.5950, Val Loss: 0.6070
Epoch 5: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 6: Train Loss: 0.5950, Val Loss: 0.6070
Epoch 7: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 8: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 9: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 10: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 11: Train Loss: 0.5950, Val Loss: 0.6070
Epoch 12: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 13: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 14: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 15: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 16: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 17: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 18: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 19: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 20: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 21: Train Loss: 0.5951, Val Loss: 0.6070
Epoch 22: Train Loss: 

In [603]:
# Testing
test_loss = 0.0
rbf.eval()  # Ensure model is in evaluation mode
with torch.no_grad():  # No gradients needed
    for inputs, targets in dl_test:
        outputs = rbf(inputs).squeeze(1)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(dl_test.dataset)
rmse = np.sqrt(test_loss)  # Calculate RMSE
print(f"Test RMSE: {rmse:.4f}")

Test RMSE: 0.7789


In [166]:
tensor = torch.rand((2,2))
tensor.shape

torch.Size([2, 2])

In [167]:
tensor.expand(3, 2, 2, 4)

RuntimeError: The expanded size of the tensor (4) must match the existing size (2) at non-singleton dimension 3.  Target sizes: [3, 2, 2, 4].  Tensor sizes: [2, 2]